In [1]:
import amici
import os 

In [2]:
biomodel_id = 'BIOMD0000000630'
model_fp = f'../biosimulator_processes/model_files/sbml/{biomodel_id}_url.xml'
sbml_importer = amici.SbmlImporter(model_fp)
model_name = biomodel_id
model_dir = "model_dir"
sbml_importer.sbml2amici(model_name, model_dir)
# load the model module
model_module = amici.import_model_module(model_name, model_dir)
# instantiate model
model = model_module.getModel()
# instantiate solver
method = model.getSolver()

In [3]:
from amici.sbml_import import get_species_initial



In [13]:
dir(model)

['__bool__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__deref__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'clone',
 'fdsigmaydy',
 'fdspline_slopesdp',
 'fdspline_valuesdp',
 'fdtotal_cldp',
 'fdtotal_cldx_rdata',
 'fdx_rdatadp',
 'fdx_rdatadtcl',
 'fdx_rdatadx_solver',
 'get',
 'getAddSigmaResiduals',
 'getAlwaysCheckFinite',
 'getAmiciCommit',
 'getAmiciVersion',
 'getExpressionIds',
 'getExpressionNames',
 'getFixedParameterById',
 'getFixedParameterByName',
 'getFixedParameterIds',
 'getFixedParameterNames',
 'getFixedParameters',
 'getInitialStateSensitivities',
 'getInitialStates',
 'getMinimumSigmaResiduals',
 'getName',
 'getObservableIds

In [4]:
floating_species_list = list(model.getStateNames())

floating_species_list

['plasminogen',
 'plasmin',
 'single intact chain urokinase-type plasminogen activator',
 'two-chain urokinase-type plasminogen activator',
 'x',
 'x-plasmin']

In [18]:
# get/set parameter:

model.getParameterNames()

('degradation constant 1',
 'degradation constant 2',
 'hill cooperativity',
 'k1',
 'parameter_8',
 'k1',
 'k2',
 'k1',
 'k1',
 'v',
 'v',
 'v')

In [19]:
# set timepoints
model.setTimepoints([0, 1])

In [20]:
rdata = amici.runAmiciSimulation(model, method)

In [21]:
rdata.x

array([[1.00000000e-02, 0.00000000e+00, 2.00000000e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.95249026e-02, 1.82020113e-06, 4.90814799e-03, 1.32035442e-13,
        0.00000000e+00, 0.00000000e+00]])

In [ ]:
model.getStateNames()

In [22]:
dir(solver)

['__bool__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__deref__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'clone',
 'computingASA',
 'computingFSA',
 'get',
 'getAbsoluteTolerance',
 'getAbsoluteToleranceB',
 'getAbsoluteToleranceFSA',
 'getAbsoluteToleranceQuadratures',
 'getAbsoluteToleranceSteadyState',
 'getAbsoluteToleranceSteadyStateSensi',
 'getConstraints',
 'getInternalSensitivityMethod',
 'getInterpolationType',
 'getLinearMultistepMethod',
 'getLinearSolver',
 'getMaxConvFails',
 'getMaxNonlinIters',
 'getMaxStepSize',
 'getMaxSteps',
 'getMaxStepsBackwardProblem',
 'getMaxTime',
 'getNewtonDampingFactorLowerBound',
 'getNewtonDampingF

In [23]:
solver

<Swig Object of type 'std::unique_ptr< amici::Solver > *' at 0x11df02a30
  reporting_mode: <RDataReporting.full: 0>
  sens_meth: <SensitivityMethod.forward: 1>
  sens_order: <SensitivityOrder.none: 0>
  sens_meth_preeq: <SensitivityMethod.forward: 1>
  maxsteps: 10000
  maxtime: 0.0s
  abs_tol: 1e-16
  rel_tol: 1e-08
  abs_tol_b: 1e-16
  rel_tol_b: 1e-08
  abs_tol_fsa: 1e-16
  rel_tol_fsa: 1e-08
  abs_tol_quad: 1e-12
  rel_tol_quad: 1e-08
  abs_tol_ss: 1e-14
  rel_tol_ss: 1e-06
  abs_tol_sss: 1e-14
  rel_tol_sss: 1e-06
  int_sens_meth: <InternalSensitivityMethod.simultaneous: 1>
  int_type: <InterpolationType.polynomial: 2>
  linsol: <LinearSolver.KLU: 9>
  lmm: <LinearMultistepMethod.BDF: 2>
  newton_damp_mode: <NewtonDampingFactorMode.on: 1>
  newton_damp_lb: 1e-08
  newton_maxsteps: 0
  newton_ss_check: False
  sens_ss_check: True
  interpolation_type: <InterpolationType.polynomial: 2>
  ism: <InternalSensitivityMethod.simultaneous: 1>
  nlsol_iter: <NonlinearSolverIteration.newton:

In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


In [2]:
#### Step 1: Define the document to be read by the Composite, which is implemented by the Process.
biomodel_id = 'BIOMD0000000630'
model_fp = f'../biosimulator_processes/model_files/sbml/{biomodel_id}_url.xml'
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30

comparison_document = ComparisonDocument(
    simulators=['copasi', 'amici'],
    duration=20,
    num_steps=32,
    model_filepath=model_fp
)


comparison_document.composite

{'copasi_0': {'_type': 'process',
  'address': 'local:copasi',
  'config': {'model': {'model_source': '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'}},
  'inputs': {'floating_species_concentrations': ['floating_species_concentrations_store'],
   'model_parameters': ['model_parameters_store'],
   'time': ['time_store'],
   'reactions': ['reactions_store']},
  'outputs': {'floating_species_concentrations': ['floating_species_concentrations_store'],
   'time': ['time_store']}},
 'amici_1': {'_type': 'process',
  'address': 'local:amici',
  'config': {'model': {'model_source': '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'}},
  'inputs': {'floating_species_concentrations': ['floating_species_concentrations_store'],
   'model_parameters': ['model_parameters_store'],
   'time': ['time_store'],
   'reactions': ['reactions_store']},
  'outputs': {'floating_species_concentrations': ['floating_species_concentrations_store'],
   'time': ['time_store']}}

In [6]:
from amici.sbml_import import get_species_initial
floating_species_initial = [get_species_initial(mol_id) for mol_id in floating_species_list]

AttributeError: 'str' object has no attribute 'isSetInitialConcentration'